In [2]:
"""
The script demonstrates a simple example of using ART with PyTorch. The example train a small model on the MNIST dataset
and creates adversarial examples using the Fast Gradient Sign Method. Here we use the ART classifier to train the model,
it would also be possible to provide a pretrained model to the ART classifier.
The parameters are chosen for reduced computational requirements of the script and not optimised for accuracy.
"""
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from art.attacks import FastGradientMethod
from art.classifiers import PyTorchClassifier
from art.utils import load_mnist



In [ ]:
# Step 0: Define the neural network model, return logits instead of activation in forward method
#
#class Net(nn.Module):
#    def __init__(self):
#        super(Net, self).__init__()
#        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1)
#        self.conv_2 = nn.Conv2d(in_channels=4, out_channels=10, kernel_size=5, stride=1)
#        self.fc_1 = nn.Linear(in_features=4 * 4 * 10, out_features=100)
#        self.fc_2 = nn.Linear(in_features=100, out_features=10)
#
#    def forward(self, x):
#        x = F.relu(self.conv_1(x))
#        x = F.max_pool2d(x, 2, 2)
#        x = F.relu(self.conv_2(x))
#        x = F.max_pool2d(x, 2, 2)
#        x = x.view(-1, 4 * 4 * 10)
#        x = F.relu(self.fc_1(x))
#        x = self.fc_2(x)
#        return x
#

numberOfClasses = 2

class Net(nn.Module):
    def __init__(self, num_classes=numberOfClasses):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(-1, 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [ ]:
# Step 1: Load the dataset
yalefaces = datasets.ImageFolder(root ='/Users/floriannuding/Desktop/Masterarbeit_Lokal/Datensätze_ready/Datensätze_ready/yalefaces_reworked', 
                                 transform=data_transform)

dataset_loader = torch.utils.data.DataLoader(yalefaces,
                                             batch_size=len(yalefaces))

dataiter = iter(dataset_loader)
images, labels = dataiter.next()


x_train = images.data.numpy()

y_train = np.zeros((len(labels.data.numpy()), len(yalefaces.classes)))
y_train[np.arange(len(labels.data.numpy())), labels.data.numpy()] = 1


x_test = "TODO"
y_test = "TODO"


In [ ]:
# Step 2: Create the model

model = Net()

In [ ]:
# Step 2a: Define the loss function and the optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
# Step 3: Create the ART classifier

classifier = PyTorchClassifier(model=model, loss=criterion, optimizer=optimizer, input_shape=(3, 224, 224), nb_classes=numberOfClasses)

In [ ]:
# Step 4: Train the ART classifier

classifier.fit(x_train, y_train, batch_size=10, nb_epochs=3)

In [ ]:
# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

In [ ]:
# Step 6: Generate adversarial test examples
#attack = FastGradientMethod(classifier=classifier, eps=0.2)
attack = UniversalPerturbation(classifier, attacker='deepfool', attacker_params=None, delta=0.2, max_iter=20, eps=10.0, norm=inf)

x_test_adv = attack.generate(x=x_test)

In [ ]:
# Step 7: Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

In [150]:
from torchvision import datasets, transforms
import torch
import numpy as np

data_transform = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),                     
        transforms.Normalize(                     
            mean=[0.485, 0.456, 0.406],               
            std=[0.229, 0.224, 0.225]                  
        )])

def my_numpy_loader(filename):
    return np.load(filename)


yalefaces = datasets.ImageFolder(root ='/Users/floriannuding/Desktop/Masterarbeit_Lokal/Datensätze_ready/Datensätze_ready/yalefaces_reworked', 
                                 transform=data_transform)

dataset_loader = torch.utils.data.DataLoader(yalefaces,
                                             batch_size=len(yalefaces))

dataiter = iter(dataset_loader)
images, labels = dataiter.next()


x_train = images.data.numpy()

y_train = np.zeros((len(labels.data.numpy()), len(yalefaces.classes)))
y_train[np.arange(len(labels.data.numpy())), labels.data.numpy()] = 1


(60000, 10)

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

(103, 3, 224, 224)

In [89]:
dataiter = iter(dataset_loader)
images, labels = dataiter.next()

images.__class__

torch.Tensor

In [130]:
dir(labels.data)

['__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ilshift__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rfloordiv__',
 '__rmul__',
 '__rpow__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__setstate_

In [132]:
len(yalefaces.classes)

13